In [1]:
import numpy as np
from libow8 import sensor_net
import matplotlib.pyplot as plt
import owutils as ut
from designs_diag import designs, align_receiver_to_transmitter
from panel_ow import Panel
from pyswarms.single.global_best import GlobalBestPSO

In [ ]:
h_ww = None
r_sens = None
n_particles =40


# ----------------------
# Parameters
# ----------------------
KEY = 'A'
params_d = designs[KEY]
params_amb = designs['A'].copy()
params_amb['r_master'] = params_d['r_lights']
params_amb['PT_master'] = params_d['PT_lights']
h_ww = None
h_amb = None

# ----------------------
# Simulation function
# ----------------------
def sensor_ar(x, params_d=None): 
    """
    Runs the optical wireless simulation for a set of particles.
    x shape: (n_particles, 3) -> [theta, phi, area]
    """
    global h_ww
    global r_sens
    global nS_sens
    global n_particles
    global h_amb

    theta = x[:,0]
    phi   = x[:,1]
    area  = x[:,2]

    r_sensor = np.tile(r_sens, (n_particles, 1)) 
    nS_sensor = nS_sens 
    nR = ut.spher_to_cart_ar(1, theta, phi).T

    params_amb['r_sensor'] = r_sensor
    params_amb['nR_sensor'] = nR
    params_amb['A_sensor'] = area   # pass area into simulation
    params_amb['nS_sensor'] = nS_sensor   

    l_amb = sensor_net(**params_amb) 
    l_amb.calch(h_ww=h_amb)
    l_amb.light_sim()
    h_amb = l_amb.h_ww

    params_d['r_sensor'] = r_sensor
    params_d['nR_sensor'] = nR
    params_d['A_sensor'] = area   
    params_d['nS_sensor'] = nS_sensor  
    #print(params_d)
    
 
    l = sensor_net(**params_d) 
    l.calch(h_ww = h_ww)
    l.light_sim()
    h_ww = l.h_ww
    l.calc_noise() #useless
    l.calc_rq() 



    #connect
    p_all = np.sum(np.sum(l_amb.Pin_sm_diff,axis = 0),axis = 1) + np.sum(l_amb.Pin_sm,axis = 0) +l_amb.Pin_sa + 0.5*l.Pin_sm_tot.flatten() #LOS + Diffuse + Ambient
    p_no_sun = np.sum(np.sum(l_amb.Pin_sm_diff,axis = 0),axis = 1) + np.sum(l_amb.Pin_sm,axis = 0) 
    p_los = np.sum(l_amb.Pin_sm,axis = 0) 
    p_diff = np.sum(np.sum(l_amb.Pin_sm_diff,axis = 0),axis = 1)
    p_amb = l_amb.Pin_sa
    G_all = p_all/l_amb.A_sensor
    G_los = p_los/l_amb.A_sensor
    G_ac = l.Pin_sm_tot/l_amb.A_sensor
    G_ac = G_ac.flatten()
    G_no_sun = p_no_sun/l_amb.A_sensor

    panels = []
    pmax = np.zeros(G_los.shape)
    ind  = np.zeros(G_los.shape, dtype=int)
    bw = np.zeros(G_los.shape)
    snr = np.zeros(G_los.shape)
    signal = np.zeros(G_los.shape)
    noise = np.zeros(G_los.shape)
    snr_dB = np.zeros(G_los.shape)
    v = np.zeros(G_los.shape)
    C = np.zeros(G_los.shape)
    req = np.zeros(G_los.shape)
    rx = np.zeros(G_los.shape)

    freq = np.linspace(100,20000,400)

    for i in range(p_all.shape[0]):
        panels.append(Panel(l_amb.A_sensor[i]*1e4, rs=1, rsh=1000, n=1.6, voc=0.64, isc=35e-3, G=G_all[i], Gac = G_ac[i]))
        panels[i].run(False)
        pmax[i] = panels[i].Pmax
        ind[i] = int(panels[i].ind)    
        panels[i].calc_capacitance()
        panels[i].set_circuit(Rc = 10, Lo = 10, Co = 220e-6)
        panels[i].find_bw()    
        bw[i] = panels[i].BW[int(ind[i])]
        panels[i].tf(freq)
        panels[i].thermal_noise()
        panels[i].all_thermal_noise(freq)
        panels[i].shot_noise(freq)
        panels[i].vp2p(freq)
        signal[i] = panels[i].vac[int(ind[i])]
        noise[i] = 4 *(panels[i].th_noise[int(ind[i])] + panels[i].sh_noise[int(ind[i])])
        snr[i] = signal[i]**2/noise[i]
        snr_dB[i] = 10*np.log10(snr[i])
        v[i] = panels[i].V[int(ind[i])]
        C[i] = panels[i].C[int(ind[i])]
        req[i] = panels[i].req[int(ind[i])]
        rx = panels[i].rx
    l.calc_tbattery(br = np.floor(bw)*0.4)



    e_day = pmax*3600*4*0.8
    V = 3.3 #Volt
    cycle_p = np.array([c.calc_cycle_consumption() for c in l.cycles]) * V
    day_s = 3600*24
    cycles_day = day_s/l.Tcycle
    e_day_c = cycles_day * cycle_p

    penalty = np.zeros(e_day_c.shape)

    penalty[snr_dB<8.5] = 10
    penalty[e_day < e_day_c] = 1e-2* (e_day_c[e_day < e_day_c] - e_day[e_day < e_day_c])
    
    
    fitness = area + penalty

    return fitness


# ----------------------
# Fitness wrapper for PSO
# ----------------------
def fit_function(x):
    """
    Normalized swarm values x in [0,1]^3
    Columns: theta, phi, area
    """
    # Denormalize
    theta = x[:,0] * (np.pi/2)
    phi   = x[:,1] * (2*np.pi)
    area  = 0.0001 + x[:,2] * (0.008 - 0.0001)  

    # Rebuild for simulation
    x_denorm = np.column_stack((theta, phi, area))

    f = sensor_ar(x_denorm, params_d=params_d)
    g = np.array(f).reshape(x.shape[0])
    return g


# ----------------------
# Main optimization loop
# ----------------------
r_sen = designs[KEY]['r_sensor']
nS_sen = np.round(align_receiver_to_transmitter(r_sen, np.array([5,5,3])), 2)
N = r_sen.shape[0]

pos_l = [0]*N   # Sensor positions
pow_l = [0]*N   # Optical powers
op_l = [0]*N    # Optimal orientations (theta, phi, area)

options = {'c1': 0.5, 'c2': 0.3, 'w': 0.7}

# Normalized bounds [0,1] for all parameters
lb = np.zeros(3)
ub = np.ones(3)

all_cost_histories = [] 

for i in range(0, N):
    r_sens = r_sen[i]
    nS_sens = nS_sen[i]
    print("Optimizing particle " + str(i+1) + " of " + str(N))

    optimizer = GlobalBestPSO(
        n_particles=n_particles,
        dimensions=3,   # theta, phi, area
        options=options,
        bounds=(lb, ub)
    )

    best_cost, best_pos = optimizer.optimize(fit_function, iters=200)

    # Denormalize best_pos
    theta = best_pos[0] * (np.pi/2)
    phi   = best_pos[1] * (2*np.pi)
    area  = 0.0001 + best_pos[2] * (0.008 - 0.0001)
    pos_l[i] = r_sens
    op_l[i]  = [theta, phi, area]
    pow_l[i] = best_cost

    all_cost_histories.append(optimizer.cost_history)


# ----------------------
# Results
# ----------------------
print("\nOptimization complete!")
for i in range(N):
    print(f"Sensor {i+1}: Position={pos_l[i]}, Power={pow_l[i]:.4f}, Orientation={op_l[i]}")

2025-10-16 13:02:14,973 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.7}


Optimizing particle 1 of 21


pyswarms.single.global_best:   0%|                                        |0/200/home/alexandros/Έγγραφα/GitHub/eh-ow (working now)/libow8.py:1133: RuntimeWarning: divide by zero encountered in log10
  self.snr_s_los_dB = 10 * np.log10(self.snr_s_los)
/home/alexandros/Έγγραφα/GitHub/eh-ow (working now)/libow8.py:1209: RuntimeWarning: divide by zero encountered in divide
  self.PT_rq_m_los = 2 * self.g0 * np.sqrt( self.n_s ) / self.hi_sm_los
/home/alexandros/anaconda3/lib/python3.11/site-packages/numpy/lib/polynomial.py:780: RuntimeWarning: invalid value encountered in multiply
  y = y * x + pv
pyswarms.single.global_best: 100%|█████████████████████|200/200, best_cost=0.132
2025-10-16 13:34:40,360 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.13213610426215489, best pos: [0.66720621 0.25174757 0.99981642]
2025-10-16 13:34:40,365 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.7}


Optimizing particle 2 of 21


pyswarms.single.global_best: 100%|███████████████████|200/200, best_cost=0.00352
2025-10-16 14:06:25,915 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0035210799316160303, best pos: [0.72804836 0.28523483 0.43304809]
2025-10-16 14:06:25,922 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.7}


...

pyswarms.single.global_best: 100%|██████████████████|200/200, best_cost=0.000555
2025-10-16 15:54:49,589 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0005553370629663922, best pos: [0.74388938 0.3637573  0.0576376 ]
2025-10-16 15:54:49,595 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.7}


Optimizing particle 6 of 21


pyswarms.single.global_best: 100%|██████████████████|200/200, best_cost=0.000389
2025-10-17 10:06:40,305 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.0003888458281363048, best pos: [0.76859147 0.39525613 0.03656276]
2025-10-17 10:06:40,312 - pyswarms.single.global_best - INFO - Optimize for 200 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.7}


Optimizing particle 7 of 21


pyswarms.single.global_best:  76%|█████████████▋    |152/200, best_cost=0.000308

In [ ]:
plt.figure(figsize=(10,6))
for i, cost_history in enumerate(all_cost_histories):
    plt.plot(cost_history, linewidth=2, label=f"Sensor {i+1}")
plt.xlabel("Iteration")
plt.ylabel("Best Cost (1/Power)")
plt.title("PSO Convergence Curves for All Sensors")
plt.grid(True, linestyle="--", alpha=0.7)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
pos_l = np.array(pos_l)   
pow_l = np.array(pow_l)   
op_l  = np.array(op_l)    

all_cost_histories = np.array(all_cost_histories, dtype=object)

results = {
    "positions": pos_l,        # sensor positions
    "orientations": op_l,      # best [theta, phi, area]
    "powers": pow_l,           # fitness / best cost
    "cost_histories": all_cost_histories,
}

np.savez("SUN_TO_USE_22.npz", **results)
print("\nResults saved")

In [28]:
op_l[:,2]

array([0.0049429 , 0.00174434, 0.00082035, 0.00045944, 0.00029041,
       0.00019419, 0.00015546, 0.00013425, 0.00013089, 0.00014398,
       0.00015349, 0.00018459, 0.00023544, 0.00031838, 0.00044112,
       0.00062167, 0.00087225, 0.00120623, 0.00164714, 0.00218577,
       0.00287589])

In [7]:
op_l[:,0]/np.pi

array([0.33805325, 0.34536111, 0.37182189, 0.31521576, 0.25605432,
       0.40097618, 0.32848711, 0.35102927, 0.45564686, 0.40982024,
       0.43515509, 0.38125764, 0.41556377, 0.37435949, 0.41708226,
       0.38273215, 0.4246896 , 0.43063024, 0.43158257, 0.41853035,
       0.43502265])

In [8]:
nor_arr = ut.spher_to_cart_ar(1, op_l[:,0],op_l[:,1]).T

In [10]:
nor_arr

array([[ 0.01258183,  0.87325332,  0.48710404],
       [-0.16890873,  0.86801446,  0.4669269 ],
       [-0.41166726,  0.82277181,  0.39188852],
       [-0.35120127,  0.75885012,  0.54845616],
       [-0.45585658,  0.55786235,  0.69353037],
       [-0.75249758,  0.58313881,  0.30609888],
       [-0.67233731,  0.53353873,  0.51312666],
       [-0.7963502 ,  0.40290052,  0.45110701],
       [-0.97994049,  0.14292121,  0.13888904],
       [-0.84413891,  0.45748289,  0.27953336],
       [-0.96991943, -0.1353769 ,  0.20231016],
       [-0.92980676,  0.05135102,  0.36444815],
       [-0.94825205, -0.17913108,  0.26216429],
       [-0.90380017, -0.1878109 ,  0.38454171],
       [-0.93495359, -0.24397913,  0.25755768],
       [-0.88651827, -0.29050192,  0.36013052],
       [-0.91368734, -0.33201679,  0.23439345],
       [-0.90608704, -0.36367469,  0.21621055],
       [-0.89709018, -0.38695893,  0.21328851],
       [-0.87625683, -0.40998097,  0.2531592 ],
       [-0.87704339, -0.43554616,  0.202